# Experimentación métodos iterativos

In [11]:
import os
import numpy as np
import subprocess as sp

In [12]:
dir = "../data"

## Funciones auxiliares

In [13]:
def generar_sistema_edd(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    suma = np.sum(np.abs(A), axis=1)
    A += np.diag(suma)

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_edd_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_simetrico(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    A = A.T @ A

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_sim_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_triangular(tam, sol, tipo):
    A = np.tril(np.random.randint(1, 40, (tam, tam)))
    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_triang_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

In [14]:
def correr_algoritmo(dataset, tipo, tam, metodo, iteraciones, tol):        
    proceso = sp.run(["../src/iterativo", f"{dir}/{dataset}/matriz_{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)


In [15]:
def crear_archivos(n, low, high, tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        x = np.random.randint(low, high, tam)
        
        generar_sistema_edd(tam, x, tipo)
        generar_sistema_simetrico(tam, x, tipo)
        generar_sistema_triangular(tam, x, tipo)

        np.savetxt(f"{dir}/{tipo}/vector_x_{tam}.txt", x, "%d", " ")

## Generar instancias

In [16]:
n = 8

### Vector solucion chico

In [17]:
crear_archivos(n, 0, 10, "chico")

### Vector solucion mediano

In [18]:
crear_archivos(n, 50, 100, "mediano")

### Vector solucion grande

In [19]:
crear_archivos(n, 200, 300, "grande")

## Correr algoritmos

In [20]:
# res = []
# correr_algoritmo("chico", "LU", 1000, 1e-20)

-Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
-Tiempo final de computo por iteraciones y por tamaño de matrices
-LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
-Comparar LU con todos los metodos iterativos.

## Generación de Gráficos

### Error

In [21]:
def cargar_soluciones(tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir}/{tipo}/vector_x_{tam}.txt"))

In [30]:
skipped = 0
datasets = ["chico", "mediano", "grande"]
algoritmos = ["J", "JS", "GS", "GSS"]
tipos = ["edd", "sim", "triang"]
max_iter = 500
step = 50
tolerancia = 1e-10
resultados = {}

for ds in datasets:
    for algoritmo in algoritmos:
        resultados[f"{algoritmo}"] = []
        for tipo in tipos:
            for i in range(2, n + 1):
                for j in range(1, max_iter + 1, step):
                    # Aca va el timeit
                    res = correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    resultados[f"{algoritmo}"].append({f"{algoritmo}|{tipo}|{2**i}|{j}":res})

# FALTA LU
for ds in datasets:
    resultados["LU"] = []
    for tipo in tipos:
        for i in range (2, n + 1):
            res = correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            resultados["LU"].append({f"LU|{tipo}|{2**i}":res})
resultados

{'J': [{'edd|4|1': array([408.983, 386.511, 386.187, 370.478])},
  {'edd|4|51': array([286., 265., 267., 225.])},
  {'edd|4|101': array([286., 265., 267., 225.])},
  {'edd|4|151': array([286., 265., 267., 225.])},
  {'edd|4|201': array([286., 265., 267., 225.])},
  {'edd|4|251': array([286., 265., 267., 225.])},
  {'edd|4|301': array([286., 265., 267., 225.])},
  {'edd|4|351': array([286., 265., 267., 225.])},
  {'edd|4|401': array([286., 265., 267., 225.])},
  {'edd|4|451': array([286., 265., 267., 225.])},
  {'edd|8|1': array([431.578, 416.435, 489.38 , 519.976, 463.126, 536.776, 457.437,
          498.883])},
  {'edd|8|51': array([219.006, 229.005, 282.006, 281.006, 243.006, 298.006, 272.005,
          289.006])},
  {'edd|8|101': array([219., 229., 282., 281., 243., 298., 272., 289.])},
  {'edd|8|151': array([219., 229., 282., 281., 243., 298., 272., 289.])},
  {'edd|8|201': array([219., 229., 282., 281., 243., 298., 272., 289.])},
  {'edd|8|251': array([219., 229., 282., 281., 243.